In [207]:
#Data
import numpy as np
import pandas as pd

# Model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

# Metricas
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import plot_confusion_matrix

# Plot
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

## Idem MOdel.py, pero quitando columna EmploymentType


Carga de Datos y Filtrado de Datos

In [208]:
train = pd.read_csv("travel_insurance_prediction_train.csv")
test = pd.read_csv("travel_insurance_prediction_test.csv")

train["Employment Type"] = train["Employment Type"].replace(["Private Sector/Self Employed"], 0)
train["Employment Type"] = train["Employment Type"].replace(["Government Sector"], 1)
test["Employment Type"] = test["Employment Type"].replace(["Private Sector/Self Employed"], 0)
test["Employment Type"] = test["Employment Type"].replace(["Government Sector"], 1)

train["GraduateOrNot"] = train["GraduateOrNot"].replace(["Yes"], 0)
train["GraduateOrNot"] = train["GraduateOrNot"].replace(["No"], 1)
test["GraduateOrNot"] = test["GraduateOrNot"].replace(["Yes"], 0)
test["GraduateOrNot"] = test["GraduateOrNot"].replace(["No"], 1)

train["FrequentFlyer"] = train["FrequentFlyer"].replace(["Yes"], 0)
train["FrequentFlyer"] = train["FrequentFlyer"].replace(["No"], 1)
test["FrequentFlyer"] = test["FrequentFlyer"].replace(["Yes"], 0)
test["FrequentFlyer"] = test["FrequentFlyer"].replace(["No"], 1)

train["EverTravelledAbroad"] = train["EverTravelledAbroad"].replace(["Yes"], 0)
train["EverTravelledAbroad"] = train["EverTravelledAbroad"].replace(["No"], 1)
test["EverTravelledAbroad"] = test["EverTravelledAbroad"].replace(["Yes"], 0)
test["EverTravelledAbroad"] = test["EverTravelledAbroad"].replace(["No"], 1)

train = train.drop(columns=["Customer", "Employment Type"])

In [209]:
suma2 = train["TravelInsurance"].sum()
suma2

532

In [210]:

trainfilter = train["AnnualIncome"]>500000
trainMoreIncome = train[trainfilter]
trainMoreIncome.describe()

,Age,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad,TravelInsurance
count,1204.000000,1204.000000,1.204000e+03,1204.000000,1204.000000,1204.000000,1204.000000,1204.000000
mean,29.750831,0.140365,1.051578e+06,4.748339,0.275748,0.737542,0.764120,0.406977
std,2.988096,0.347510,3.137100e+05,1.610116,0.447076,0.440153,0.424724,0.491475
min,25.000000,0.000000,5.500000e+05,2.000000,0.000000,0.000000,0.000000,0.000000
25%,28.000000,0.000000,8.000000e+05,4.000000,0.000000,0.000000,1.000000,0.000000
50%,29.000000,0.000000,1.050000e+06,5.000000,0.000000,1.000000,1.000000,0.000000
75%,33.000000,0.000000,1.300000e+06,6.000000,1.000000,1.000000,1.000000,1.000000
max,35.000000,1.000000,1.800000e+06,9.000000,1.000000,1.000000,1.000000,1.000000


In [211]:
suma = trainMoreIncome["TravelInsurance"].sum()
suma

490

Aqui me surgio la duda sobre si esta bien reemplazar las palabras por numeros 0 y 1, o deberiamos aplicar un one hot encoder. Las variables son categoricas son binarias. 

In [212]:
X, y = train.iloc[:, 0:7], train.TravelInsurance

In [213]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=0)

In [214]:
X_train.shape
X_test.shape

(1192, 7)

-----------------------------------------------------------------------------------------------
# SGD Classifier

In [215]:
from sklearn.preprocessing import RobustScaler, MinMaxScaler, Normalizer, QuantileTransformer, PowerTransformer
clf = make_pipeline(RobustScaler(),
                   SGDClassifier(random_state=42))
clf.fit(X_train, y_train)

Pipeline(steps=[('robustscaler', RobustScaler()),
                ('sgdclassifier', SGDClassifier(random_state=42))])

### Metricas en Entrenamiento

In [216]:
y_pred_train = clf.predict(X_train)
print('Accuracy Score: ', accuracy_score(y_train, y_pred_train))
print('Precision Score: ', precision_score(y_train, y_pred_train))
print('F1 Score: ', f1_score(y_train, y_pred_train))
print('Recall Score: ', recall_score(y_train, y_pred_train))

Accuracy Score:  0.7550335570469798
Precision Score:  0.6464646464646465
F1 Score:  0.6368159203980099
Recall Score:  0.6274509803921569


### Metricas en Test

In [217]:
y_pred_test = clf.predict(X_test)
print('Accuracy Score: ', accuracy_score(y_test, y_pred_test))
print('Precision Score: ', precision_score(y_test, y_pred_test))
print('F1 Score: ', f1_score(y_test, y_pred_test))
print('Recall Score: ', recall_score(y_test, y_pred_test))

Accuracy Score:  0.7256711409395973
Precision Score:  0.6223277909738717
F1 Score:  0.6157461809635723
Recall Score:  0.6093023255813953


Habria que analizar porque esta dando una accuracy mejor en test que en el mismo entrenamiento. Puede ser por los pocos datos que tenemos?

In [218]:
# Selección de parámetros
param_grid = {
    'loss': ['perceptron','hinge','log','squared_loss','epsilon_insensitive'],
    'penalty' : ['l2', 'l1', 'none'],
    'alpha' : [0.0001, 0.001, 0.01, 0.1],
    'random_state': [42],
    'learning_rate': ['optimal','constant','invscaling'],
    'eta0': [0.0001, 0.001, 0.01, 0.1]
}

best_clf = make_pipeline(StandardScaler(), GridSearchCV(SGDClassifier(), param_grid, cv=3))
best_clf.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('gridsearchcv',
                 GridSearchCV(cv=3, estimator=SGDClassifier(),
                              param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1],
                                          'eta0': [0.0001, 0.001, 0.01, 0.1],
                                          'learning_rate': ['optimal',
                                                            'constant',
                                                            'invscaling'],
                                          'loss': ['perceptron', 'hinge', 'log',
                                                   'squared_loss',
                                                   'epsilon_insensitive'],
                                          'penalty': ['l2', 'l1', 'none'],
                                          'random_state': [42]}))])

### Metricas en Entrenamiento

In [155]:
y_pred_train = best_clf.predict(X_train)
print('Accuracy Score: ', accuracy_score(y_train, y_pred_train))
print('Precision Score: ', precision_score(y_train, y_pred_train))
print('F1 Score: ', f1_score(y_train, y_pred_train))
print('Recall Score: ', recall_score(y_train, y_pred_train))

Accuracy Score:  0.7885906040268457
Precision Score:  0.8095238095238095
F1 Score:  0.6181818181818182
Recall Score:  0.5


### Metricas en Test

In [156]:
y_pred_test = best_clf.predict(X_test)
print('Accuracy Score: ', accuracy_score(y_test, y_pred_test))
print('Precision Score: ', precision_score(y_test, y_pred_test))
print('F1 Score: ', f1_score(y_test, y_pred_test))
print('Recall Score: ', recall_score(y_test, y_pred_test))

Accuracy Score:  0.7659395973154363
Precision Score:  0.7745454545454545
F1 Score:  0.6042553191489362
Recall Score:  0.49534883720930234


El tema de este ultimo modelo es que no sabemos que hiperparametros adopto por el pipeline. Pero como sólo tenemos que cargar los datos de prediccion de test podria funcionar

-----------------------------------------------------------------------------------------------
# Decision Tree

## Decision Tree Vanilla

In [157]:
tree_def = DecisionTreeClassifier(random_state=42)
tree_def.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [158]:
y_pred = tree_def.predict(X_train)
print('Accuracy Score: ', accuracy_score(y_train, y_pred))
print('Precision Score: ', precision_score(y_train, y_pred))
print('F1 Score: ', f1_score(y_train, y_pred))
print('Recall Score: ', recall_score(y_train, y_pred))

Accuracy Score:  0.9664429530201343
Precision Score:  1.0
F1 Score:  0.9484536082474228
Recall Score:  0.9019607843137255


In [159]:
y_pred_test = tree_def.predict(X_test)
print('Accuracy Score: ', accuracy_score(y_test, y_pred_test))
print('Precision Score: ', precision_score(y_test, y_pred_test))
print('F1 Score: ', f1_score(y_test, y_pred_test))
print('Recall Score: ', recall_score(y_test, y_pred_test))

Accuracy Score:  0.7390939597315436
Precision Score:  0.6420047732696897
F1 Score:  0.6336866902237926
Recall Score:  0.6255813953488372


En entrenamiento avanza tanto que divide todos los datos.

In [175]:

treeWithRobustScaler = make_pipeline(RobustScaler(),
                   DecisionTreeClassifier(random_state=42))
treeWithRobustScaler.fit(X_train, y_train)

Pipeline(steps=[('robustscaler', RobustScaler()),
                ('decisiontreeclassifier',
                 DecisionTreeClassifier(random_state=42))])

In [176]:
y_pred = treeWithRobustScaler.predict(X_train)
print('Accuracy Score: ', accuracy_score(y_train, y_pred))
print('Precision Score: ', precision_score(y_train, y_pred))
print('F1 Score: ', f1_score(y_train, y_pred))
print('Recall Score: ', recall_score(y_train, y_pred))

Accuracy Score:  0.9664429530201343
Precision Score:  1.0
F1 Score:  0.9484536082474228
Recall Score:  0.9019607843137255


In [177]:
y_pred_test = treeWithRobustScaler.predict(X_test)
print('Accuracy Score: ', accuracy_score(y_test, y_pred_test))
print('Precision Score: ', precision_score(y_test, y_pred_test))
print('F1 Score: ', f1_score(y_test, y_pred_test))
print('Recall Score: ', recall_score(y_test, y_pred_test))

Accuracy Score:  0.7390939597315436
Precision Score:  0.640661938534279
F1 Score:  0.6354044548651818
Recall Score:  0.6302325581395349


## BEst Decision Tree

In [173]:
# Selección de parámetros
param_grid = {
    'criterion': ['entropy','gini'],
    'max_depth': [2,5,10,50],
    'min_samples_leaf' : [1, 2,3,8,20,40,60]
}

best_tree = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=5)
best_tree.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['entropy', 'gini'],
                         'max_depth': [2, 5, 10, 50],
                         'min_samples_leaf': [1, 2, 3, 8, 20, 40, 60]})

In [161]:
y_pred = best_tree.predict(X_train)
print('Accuracy Score: ', accuracy_score(y_train, y_pred))
print('Precision Score: ', precision_score(y_train, y_pred))
print('F1 Score: ', f1_score(y_train, y_pred))
print('Recall Score: ', recall_score(y_train, y_pred))

Accuracy Score:  0.8624161073825504
Precision Score:  0.8505747126436781
F1 Score:  0.783068783068783
Recall Score:  0.7254901960784313


In [162]:
print("El mejor score promedio cruzado validado",best_tree.best_score_)
print("-------------------------------------------------------------------------------------------------")
print("El mejor estimador es", best_tree.best_estimator_)
print("-------------------------------------------------------------------------------------------------")
print("Los parámetros que dieron los mejores resultados", best_tree.best_params_)

El mejor score promedio cruzado validado 0.8493220338983052
-------------------------------------------------------------------------------------------------
El mejor estimador es DecisionTreeClassifier(max_depth=10, min_samples_leaf=8)
-------------------------------------------------------------------------------------------------
Los parámetros que dieron los mejores resultados {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 8}


In [163]:
y_pred_test = best_tree.predict(X_test)
print('Accuracy Score: ', accuracy_score(y_test, y_pred_test))
print('Precision Score: ', precision_score(y_test, y_pred_test))
print('F1 Score: ', f1_score(y_test, y_pred_test))
print('Recall Score: ', recall_score(y_test, y_pred_test))

Accuracy Score:  0.7869127516778524
Precision Score:  0.7444444444444445
F1 Score:  0.6784810126582278
Recall Score:  0.6232558139534884


In [164]:
from sklearn.ensemble import AdaBoostRegressor

regr = AdaBoostRegressor(random_state=0, n_estimators=100)
regr.fit(X_train, y_train)
AdaBoostRegressor(n_estimators=100, random_state=0)

AdaBoostRegressor(n_estimators=100, random_state=0)

In [165]:
regr.score(X_train, y_train)

0.331532513948804

-----------------------------------------------------------------------------------------------
## SVR
El problema de este modelo es que arroja resultados continuos. Lo dejo por si encontramos que podemos modificar su salida con alguna funcion de activacion

In [166]:
from sklearn.svm import SVR

regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
regr.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svr', SVR(epsilon=0.2))])

In [167]:
y_pred = regr.predict(X_train)
y_pred

array([ 0.33562149,  0.16996008,  0.79971824,  0.66351884,  0.19969709,
        0.11468063,  0.18482693,  0.13346669,  0.10651731,  0.18292632,
        0.8000032 ,  0.80794934,  0.20008339,  0.12659669,  0.25150078,
        0.08619186,  0.2003332 ,  0.20022758,  0.17348619,  0.20010551,
        0.32452996,  0.1997603 ,  0.8124498 ,  0.79990414,  0.30427568,
        0.12805944,  0.52116852,  0.75026405,  0.87655528,  0.13346669,
        0.22021025,  0.18809941,  0.16762214,  0.41443447,  0.32834731,
        0.17656627,  0.19987509,  0.55301839,  0.34990844,  0.24027662,
       -0.0042996 ,  0.20022899,  0.11265562,  0.11563537,  0.32211186,
        0.15501886,  0.65011527,  0.19982402,  0.12051515,  0.20003162,
        0.79990414,  0.14380716,  0.2129419 ,  0.79968183,  0.40191143,
        0.20000346,  0.11623877,  0.15239688,  0.16438443,  0.79979576,
        0.24883947,  0.16507389,  0.13587699,  0.09822561, -0.0042996 ,
        0.20643734,  0.08839186,  0.09393832,  0.19965338,  0.20

In [168]:
y_pred = regr.predict(X_train)
print('Accuracy Score: ', accuracy_score(y_train, y_pred))
print('Precision Score: ', precision_score(y_train, y_pred))
print('F1 Score: ', f1_score(y_train, y_pred))
print('Recall Score: ', recall_score(y_train, y_pred))

ValueError: Classification metrics can't handle a mix of binary and continuous targets

-----------------------------------------------------------------------------------------------
## C-Support Vector Classification.
Es un modelo que tardo mucho en entrenar. En pc local, aprox 2 minutos con el 20 % de datos de entrenamiento (lo reduje para ello). Adjunto comentado el codigo y por los resultados creo que no vale la pena. Luego lo corro (o si alguno le da paciencia) con el entrenamiento al 80 % como esta seteado. 

In [6]:
#from sklearn import svm
#clf = svm.SVC(kernel='linear', cache_size=7000)

In [7]:
#clf = clf.fit(X_train, y_train)

In [8]:
#y_pred = clf.predict(X_train)
#print('Accuracy Score: ', accuracy_score(y_train, y_pred))
#print('Precision Score: ', precision_score(y_train, y_pred))
#print('F1 Score: ', f1_score(y_train, y_pred))
#print('Recall Score: ', recall_score(y_train, y_pred))

Accuracy Score:  0.6812080536912751
Precision Score:  0.5346534653465347
F1 Score:  0.5320197044334977
Recall Score:  0.5294117647058824



<li> Accuracy Score:  0.6812080536912751
<li> Precision Score:  0.5346534653465347
<li> F1 Score:  0.5320197044334977
<li> Recall Score:  0.5294117647058824

## Support Vector Machine

In [59]:
from sklearn import svm
clf = svm.SVC(kernel="rbf", C=1, cache_size=2000, )
clf.fit(X_train, y_train)

SVC(C=1, cache_size=2000)

In [60]:
y_pred = clf.predict(X_train)
print('Accuracy Score: ', accuracy_score(y_train, y_pred))
print('Precision Score: ', precision_score(y_train, y_pred))
print('F1 Score: ', f1_score(y_train, y_pred))
print('Recall Score: ', recall_score(y_train, y_pred))

Accuracy Score:  0.7986577181208053
Precision Score:  0.9038461538461539
F1 Score:  0.6103896103896104
Recall Score:  0.46078431372549017


In [61]:
#Este no me lo corrio. Deje la compu 1 hora y no funco. 
#parameters = {"kernel":("linear", "rbf"), "C":[1, 10]}
#svc = svm.SVC()
#clf = GridSearchCV(svc, parameters)
#clf.fit(X_train, y_train)

In [62]:
print("SVM con Kernel rbf, dio accuracy de 0,7986")
print("SVM con Kernel Linear, dio accuracy de 0,6812")
print("SVM con Kernel Poly, dio accuracy de 0,7885")
print("SVM con Kernel Sigmoid, dio accuracy de 0,4832")
print("SVM con Kernel rbf + gamma auto da accuracy de 1 en train, pero en test 0,63")


SVM con Kernel rbf, dio accuracy de 0,7986
SVM con Kernel Linear, dio accuracy de 0,6812
SVM con Kernel Poly, dio accuracy de 0,7885
SVM con Kernel Sigmoid, dio accuracy de 0,4832
SVM con Kernel rbf + gamma auto da accuracy de 1 en train, pero en test 0,63


In [63]:
y_pred = clf.predict(X_test)
print('Accuracy Score: ', accuracy_score(y_test, y_pred))
print('Precision Score: ', precision_score(y_test, y_pred))
print('F1 Score: ', f1_score(y_test, y_pred))
print('Recall Score: ', recall_score(y_test, y_pred))

Accuracy Score:  0.7827181208053692
Precision Score:  0.8976744186046511
F1 Score:  0.5984496124031008
Recall Score:  0.44883720930232557


In [184]:
from sklearn.preprocessing import RobustScaler, MinMaxScaler, MaxAbsScaler
clf = make_pipeline(MaxAbsScaler(), svm.SVC(kernel="sigmoid", gamma="auto"))
clf.fit(X_train, y_train)

Pipeline(steps=[('maxabsscaler', MaxAbsScaler()),
                ('svc', SVC(gamma='auto', kernel='sigmoid'))])

In [185]:
y_pred = clf.predict(X_train)
print('Accuracy Score: ', accuracy_score(y_train, y_pred))
print('Precision Score: ', precision_score(y_train, y_pred))
print('F1 Score: ', f1_score(y_train, y_pred))
print('Recall Score: ', recall_score(y_train, y_pred))

Accuracy Score:  0.7516778523489933
Precision Score:  0.75
F1 Score:  0.5316455696202532
Recall Score:  0.4117647058823529


In [186]:
y_pred = clf.predict(X_test)
print('Accuracy Score: ', accuracy_score(y_test, y_pred))
print('Precision Score: ', precision_score(y_test, y_pred))
print('F1 Score: ', f1_score(y_test, y_pred))
print('Recall Score: ', recall_score(y_test, y_pred))

Accuracy Score:  0.7533557046979866
Precision Score:  0.7931034482758621
F1 Score:  0.5558912386706948
Recall Score:  0.42790697674418604
